**Dengan menggunakan dataset anime & rating, buatlah recommendation system dengan skema berikut:**

* Gabungkan kedua data agar dapat memunculkan informasi-informasi yang ada pada dataset anime.
* Bandingkan algoritma SVD dan ALS
* Tuning algoritma yang menurut kalian lebih baik

Setelah mendapatkan model terbaik, coba prediksi rating anime berikut:

* Hunter x Hunter (2011), anime_id 11061
* Detective Conan OVA 09, anime_id 2514
* Ranma ½, anime_id 1010
* Saint Seiya: Meiou Hades Juuni Kyuu-hen, anime_id 1257 

Oleh user:

* 50
* 200
* 400
* 800

Bagaimana urutan rekomendasi yang akan kalian berikan untuk masing-masing user?


In [1]:
import pandas as pd
import numpy as np

from surprise import Reader
from surprise import Dataset

from surprise import SVD
from surprise import BaselineOnly

from surprise import accuracy
from surprise.model_selection import cross_validate, train_test_split
from surprise.model_selection import GridSearchCV

In [13]:
rating = pd.read_csv(r'C:\Users\dheof\Desktop\rating.csv')
rating

,Unnamed: 0,user_id,anime_id,rating
0,47,1,8074,10.0
1,81,1,11617,10.0
2,83,1,11757,10.0
3,101,1,15451,10.0
4,153,2,11771,10.0
...,...,...,...,...
77863,96433,999,11757,6.0
77864,96434,999,16498,9.0
77865,96435,999,21881,5.0
77866,96436,999,22319,8.0


In [14]:
anime = pd.read_csv(r'C:\Users\dheof\Desktop\anime.csv')
anime

,anime_id,name,genre,type,episodes,rating,members
0,32281,Kimi no Na wa.,"Drama, Romance, School, Supernatural",Movie,1,9.37,200630
1,5114,Fullmetal Alchemist: Brotherhood,"Action, Adventure, Drama, Fantasy, Magic, Mili...",TV,64,9.26,793665
2,28977,Gintama°,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.25,114262
3,9253,Steins;Gate,"Sci-Fi, Thriller",TV,24,9.17,673572
4,9969,Gintama&#039;,"Action, Comedy, Historical, Parody, Samurai, S...",TV,51,9.16,151266
...,...,...,...,...,...,...,...
12289,9316,Toushindai My Lover: Minami tai Mecha-Minami,Hentai,OVA,1,4.15,211
12290,5543,Under World,Hentai,OVA,1,4.28,183
12291,5621,Violence Gekiga David no Hoshi,Hentai,OVA,4,4.88,219
12292,6133,Violence Gekiga Shin David no Hoshi: Inma Dens...,Hentai,OVA,1,4.98,175


In [17]:
df = pd.merge(rating, anime, on = 'anime_id')
df

,Unnamed: 0,user_id,anime_id,rating_x,name,genre,type,episodes,rating_y,members
0,47,1,8074,10.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
1,189,3,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
2,499,5,8074,2.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
3,1299,12,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
4,1519,14,8074,6.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural",TV,12,7.46,535892
...,...,...,...,...,...,...,...,...,...,...
77863,93997,963,27909,6.0,Otome Hime,Hentai,OVA,1,6.36,1526
77864,94772,979,7549,8.0,Quiz Magic Academy: The Original Animation 2,"Comedy, Fantasy, Magic, School",OVA,1,6.24,3658
77865,95511,992,1044,4.0,Taiyou no Ouji: Horus no Daibouken,"Adventure, Drama, Magic",Movie,1,6.63,3717
77866,95884,995,2571,6.0,Mitsubachi Maya no Bouken,"Adventure, Comedy, Kids",TV,52,6.43,4426


In [18]:
df.drop(columns = ['Unnamed: 0', 'type', 'episodes', 'rating_y', 'members'], inplace = True)

In [27]:
df = df.sort_values('user_id', ascending = True)
df = df.reset_index(drop=True)
df

,user_id,anime_id,rating_x,name,genre
0,1,8074,10.0,Highschool of the Dead,"Action, Ecchi, Horror, Supernatural"
1,1,11757,10.0,Sword Art Online,"Action, Adventure, Fantasy, Game, Romance"
2,1,15451,10.0,High School DxD New,"Action, Comedy, Demons, Ecchi, Harem, Romance,..."
3,1,11617,10.0,High School DxD,"Comedy, Demons, Ecchi, Harem, Romance, School"
4,2,11771,10.0,Kuroko no Basket,"Comedy, School, Shounen, Sports"
...,...,...,...,...,...
77863,999,27899,7.0,Tokyo Ghoul √A,"Action, Drama, Horror, Mystery, Psychological,..."
77864,999,20,8.0,Naruto,"Action, Comedy, Martial Arts, Shounen, Super P..."
77865,999,4898,9.0,Kuroshitsuji,"Action, Comedy, Demons, Fantasy, Historical, S..."
77866,999,687,7.0,Tokyo Mew Mew,"Adventure, Comedy, Fantasy, Magic, Romance, Sc..."


In [38]:
user_item_rating_matrix = df.pivot_table(
    values = 'rating_x', 
    index = 'user_id', 
    columns = 'anime_id'
)

In [73]:
reader = Reader(rating_scale=(0, 10))
data = Dataset.load_from_df(df[['user_id', 'anime_id', 'rating_x']], reader)
data.df

,user_id,anime_id,rating_x
0,1,8074,10.0
1,1,11757,10.0
2,1,15451,10.0
3,1,11617,10.0
4,2,11771,10.0
...,...,...,...
77863,999,27899,7.0
77864,999,20,8.0
77865,999,4898,9.0
77866,999,687,7.0


## VALIDATE

In [42]:
trainset, testset = train_test_split(
    data,
    test_size = 0.25,
    random_state = 0
)

### SVD

In [43]:
algo = SVD()

algo.fit(trainset)
predictions = algo.test(testset)

In [44]:
accuracy.rmse(predictions)

RMSE: 1.2253


1.2253049523604054

### ALS

In [45]:
bsl_option = {
    'method' : 'als'
    }

algo = BaselineOnly(bsl_options = bsl_option)
algo.fit(trainset)
predictions = algo.test(testset)

Estimating biases using als...


In [46]:
accuracy.rmse(predictions)

RMSE: 1.2225


1.2224897982150755

RMSE TRAIN - TEST 

SVD 1.2253

ALS 1.2225

## CROSS VALIDATION

### SVD

In [47]:
algo = SVD()

cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2019  1.2031  1.2020  1.2142  1.2062  1.2055  0.0046  
MAE (testset)     0.9107  0.9128  0.9145  0.9193  0.9152  0.9145  0.0029  
Fit time          6.29    6.73    6.08    5.49    5.33    5.98    0.51    
Test time         0.21    0.16    0.23    0.21    0.15    0.19    0.03    


In [49]:
cv_svd['test_rmse'].mean()

1.2054771738424395

### ALS

In [50]:
bsl_option = {
    'method' : 'als'
    }

algo = BaselineOnly(bsl_options = bsl_option)

cv_als = cross_validate(algo, data, measures = ['RMSE', 'MAE'], cv = 5)

Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...
Estimating biases using als...


In [52]:
cv_als['test_rmse'].mean()

1.2130853847634102

RMSE CV

SVD 1.2054

ALS 1.2130

## HYPERPARAM TUNING

In [53]:
param_grid = {
    'n_epochs': [5, 10, 20], 
    'lr_all': [0.002, 0.005],
    'reg_all': [0.02, 0.4, 0.6]
}

gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)

gs.fit(data)

In [54]:
print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

1.2167752965751457
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


In [55]:
print(gs.best_score['mae'])
print(gs.best_params['mae'])

0.9236781467524467
{'n_epochs': 20, 'lr_all': 0.005, 'reg_all': 0.02}


## BEFORE AFTER TUNING

### BEFORE

In [56]:
algo = SVD()

cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2160  1.1988  1.2051  1.2022  1.2016  1.2047  0.0060  
MAE (testset)     0.9214  0.9122  0.9144  0.9054  0.9128  0.9132  0.0051  
Fit time          4.40    7.16    5.85    6.78    5.54    5.94    0.97    
Test time         0.23    0.26    0.22    0.16    0.22    0.22    0.03    


In [57]:
cv_svd['test_rmse'].mean()

1.2047369821495846

### AFTER

In [58]:
algo = SVD(
    n_epochs = 20, 
    lr_all = 0.005, 
    reg_all = 0.02
)

cv_svd = cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    1.2053  1.1967  1.2171  1.2075  1.2023  1.2058  0.0067  
MAE (testset)     0.9162  0.9094  0.9146  0.9160  0.9139  0.9140  0.0025  
Fit time          3.81    6.25    6.34    6.45    7.41    6.05    1.20    
Test time         0.10    0.32    0.14    0.25    0.15    0.19    0.08    


In [59]:
cv_svd['test_rmse'].mean()

1.2057867835827256

## PREDICTION

In [ ]:
anime id 
11061
2514
1010
1257

user
50
200
400
800

In [61]:
df_test = pd.DataFrame(columns = ['user_id','anime_id'])

for i in [50, 200, 400, 800]:
    for j in [11061, 2514, 1010, 1257]:
        df_test = df_test.append(
            {
            'user_id':i ,
            'anime_id':j
            }, 
            ignore_index = True
        )
        
df_test

,user_id,anime_id
0,50,11061
1,50,2514
2,50,1010
3,50,1257
4,200,11061
5,200,2514
6,200,1010
7,200,1257
8,400,11061
9,400,2514


## MODEL FOR RATING PREDICTION

In [93]:
# algo = SVD(
#     n_epochs = 20, 
#     lr_all = 0.005, 
#     reg_all = 0.02
# )

algo = SVD()
algo.fit(trainset)

y = []

for a, b in df_test.iterrows():
    est = algo.predict(b.user_id, b.anime_id)
    y.append(est[3])
    
df_test['rating'] = y

df_test.sort_values(by = ['user_id','rating'], ascending = [True, False], inplace = True)

### USER 50

In [94]:
df_test[df_test['user_id'] == 50]

,user_id,anime_id,rating
0,50,11061,9.513175
3,50,1257,7.923483
1,50,2514,7.746197
2,50,1010,7.746197


### USER 200

In [95]:
df_test[df_test['user_id'] == 200]

,user_id,anime_id,rating
4,200,11061,10.000000
7,200,1257,8.978361
5,200,2514,8.732287
6,200,1010,8.732287


### USER 400

In [96]:
df_test[df_test['user_id'] == 400]

,user_id,anime_id,rating
8,400,11061,8.161832
11,400,1257,6.388934
9,400,2514,6.249539
10,400,1010,6.249539


### USER 800

In [97]:
df_test[df_test['user_id'] == 800]

,user_id,anime_id,rating
12,800,11061,9.762024
13,800,2514,8.118752
14,800,1010,8.118752
15,800,1257,8.096755
